<b>Audio and Music Processing Lab - Module 2</b><br>Rafael Caro Repetto<br>rafael.caro@upf.edu<br>08.02.2023
## AMPLab2 - Introduction to music21
This notebook demonstrates the basic functionalities of music21 for processing machine readable music scores.

In [ ]:
from music21 import *

In order to visualize loaded scores using the `.show()` method, a score editor should be installed and added to [music21's user environment](http://web.mit.edu/music21/doc/usersGuide/usersGuide_24_environment.html#usersguide-24-environment). If you have no such software already installed, I strongly recommend the open source program [MuseScore](https://musescore.org/). From now on, I will be mentioning MuseScore, but you should understand your own score editor.

If the score editor was installed before installing music21, it should be automatically recognized. Otherwise, you should uncomment and run the following cell in order to add it to music21 user settings:

In [ ]:
# configure.run()

Let's load a music score. Music21 will load it as a `stream.Score`. We can give it a look using `.show()`.

In [ ]:
s = converter.parse('lseh-WeiGuoJia-HongYangDong-1.xml')
print(type(s))
s.show()

The `.elements` attribute returns a list of all the objects contained in a music21 stream.

In [ ]:
print('This score contains these {} elements'.format(len(s.elements)))
for element in s.elements:
    print('-', element)

As you can see, a `stream.Score` contains more than music. If we want to focus on the music, we can call the `.parts` attribute.
<br/>
By the way, music21 will work better if you store anything you retrieve from a stream in another stream, using the `.stream()` method.

In [ ]:
scoreParts = s.parts.stream()
for element in scoreParts.elements:
    print(element)

Now we can work with each of these parts, and have a look at them separately. We can call the `.show()` method with any stream.
<br/>
*\[When the part is opened in the score editor, notice that now all the metadata are missing, because they are not contained in the `stream.Part`, but as specific objects in the `stream.Score`.\]*

In [ ]:
part0 = scoreParts[0]
part1 = scoreParts[1]

part0.show()

These two parts should be equally long, but are they?

In [ ]:
print('Elements in part 0:', len(part0.elements))
print('Elements in part 1:', len(part1.elements))

To understand this difference, let's have a look to what is contained in each part.

In [ ]:
part0.elements

In [ ]:
part1.elements

As you can see, each part contains one instrument object, many slur objects, and many measure objects. The music is contained in the measures, so let's retrieve only those, by using the `.getElementsByClass()` method.
<br/>
Remember to store everything you retrieve from a stream in another stream.

In [ ]:
measures0 = part0.getElementsByClass(stream.Measure).stream()
measures1 = part1.getElementsByClass(stream.Measure).stream()
print('Measures in part 0:', len(measures0))
print('Measures in part 1:', len(measures1))

Let's confirm that we retrieved only measures.

In [ ]:
measures0.elements

As you can see, each measure is identifid by a number, and also an offset. The concept of **offset** is very important in music21. Every music21 object is stored in a particular position with respect to its containing stream.
<br/>
Let's check that with the first 10 measures, accessing the `.number` and `.offset` attributes in the measures.

In [ ]:
for m in measures0[0:10]:
    print('Measure number: {}\tMeasure offset: {}'.format(m.number, m.offset))

Now let's have a look to what is contained in the first measure of the first part.

In [ ]:
measure0_0 = measures0[0]
measure0_0.elements

The first measure of a part usually contains the clef, key signature and time signature objects.
<br/>
Let's have a look to what kind of information we can obtain about the key signature and the time signature.

In [ ]:
kS = measure0_0.getElementsByClass(key.KeySignature).stream()[0]
print('Class:', type(kS))
print('Number of altered pitches:', kS.sharps)
print('List of altered pitches:', kS.alteredPitches)
print('Measure number:', kS.measureNumber)
print('Offset:', kS.offset)

In [ ]:
tS = measure0_0.getElementsByClass(meter.TimeSignature).stream()[0]
print('Class:', type(tS))
print('Numerator:', tS.numerator)
print('Denominator:', tS.denominator)
print('Measure number:', tS.measureNumber)
print('Offset:', tS.offset)

Moving to another measure.

In [ ]:
measure0_3 = part0.measure(3)
print('Measure number {} in offset {}'.format(measure0_3.number, measure0_3.offset))
print()
print('It contains the following elements:')
for element in measure0_3.elements:
    print(element)

This is the most common case, measures usually contain just notes (including rests).

Now, let's start working with note objects. They also have offsets, and we can check if they are a note or a rest.

In [ ]:
n1 = measure0_3[0]
print('Class', type(n1))
print('Measure number:', n1.measureNumber)
print('Note offset:', n1.offset)
print('Is it a note?', n1.isNote)
print('Is it a rest?', n1.isRest)

Notice that the note's offset is related to its containing stream, measure 3, and not to the whole score.

A note object, when it is a note, contains a pitch object and a duration object.
<br/>
Let's have a look first to the attributes of the pitch object contained in `n1`.

In [ ]:
print('Frequency:', n1.pitch.frequency)
print('Name:', n1.pitch.name)
print('Step:', n1.pitch.step)
print('Octave:', n1.pitch.octave)
print('Name with octave:', n1.pitch.nameWithOctave)
print('Midi:', n1.pitch.midi)
print('Name in Spanish:', n1.pitch.spanish)
print()
print('Accidental:', n1.pitch.accidental.name)
print('Accidental value:', n1.pitch.accidental.alter)

Let's have a look now to the attributes of the duration object contained in `n1`.

In [ ]:
print('Duration type:', n1.duration.type)
print('Duration name:', n1.duration.fullName)
print('Duration as quarter length:', n1.duration.quarterLength)

Some of this information is very commonly used, so it can be retrieved directly from the note object.

In [ ]:
print('Name:', n1.name)
print('Step:', n1.step)
print('Octave:', n1.octave)
print('Name with octave:', n1.nameWithOctave)
print()
print('Duration as quarter length:', n1.quarterLength)

[Grace notes](https://en.wikipedia.org/wiki/Grace_note) are assigned a `quarterLength` duration of 0, and are located in the same offset as the following main note.

In [ ]:
n2 = measure0_3[1]
n3 = measure0_3[2]
print('Name:\t\t{}\t{}'.format(n2.nameWithOctave, n3.nameWithOctave))
print('Duration:\t{}\t{}'.format(n2.quarterLength, n3.quarterLength))
print('Offset:\t\t{}\t{}'.format(n2.offset, n3.offset))

Let's take now a note from the second part.

In [ ]:
measure1_9 = part1.measure(9)
n4 = measure1_9[0]
print('Name:', n4.nameWithOctave)
print('Duration:', n4.quarterLength)

With the `.lyric` attribute we can check if a note object contains lyrics.

In [ ]:
print('Does n1 have lyrics?', n1.lyric != None)
print('Does n4 have lyrics?', n4.lyric != None)

`n4` contains lyrics. Now we can look at it.

In [ ]:
print("n4's lyric:", n4.lyric)

Duration can be extended using dots and ties. Music21 handels them in the following ways.

In [ ]:
n5 = measure1_9[1]
print('Duration:', n5.quarterLength)
print('Duration type:', n5.duration.type)
print('Duration name:', n5.duration.fullName)
print('Duration as quarter length:', n5.duration.quarterLength)
print('Dots:', n5.duration.dots)

In [ ]:
measure1_20 = part1.measure(20)
measure1_21 = part1.measure(21)
n6 = measure1_20[-1]
n7 = measure1_21[2]
print('Pitch name:\t{}\t{}'.format(n6.nameWithOctave, n7.nameWithOctave))
print('Duration:\t{}\t{}'.format(n6.quarterLength, n7.quarterLength))
print('Tie:\t\t{}\t{}'.format(n6.tie.type, n7.tie.type))